# Automated Transient Determination

In [ ]:
from __future__ import division, print_function, unicode_literals
import time
#import os
#import glob
#import subprocess
import numpy as np
import math
import astropy.io.fits as fits
import sys
import datetime
import ephem

### Here we can add the below process to the existing Transients code

In [ ]:
#Creates a segmentation map from the master file. This will be used to generate flag files later.
#To tweak the sensitivity of this segmentation map (brightness of objects it identifies for flagging), change
#the DETECT_THRESH and ANALYSIS_THRESH values in segmentation.sex, located in the config_files directory.
#This file is a temporary file, but the code will not delete it by default, as it can be a useful analysis tool.

subprocess.call('sex '+master_file[0]+' -c ./config_files/segmentation.sex -WEIGHT_IMAGE '+master_weight[0], shell=True)
subprocess.call('mv ./Processed_Images/Temp/check.fits ./Processed_Images/Temp/segmentation_map_ref.fits', shell=True)
master_segmentation = ['./Processed_Images/Temp/segmentation_map_ref.fits']

In [ ]:
#This loop creates all the difference images and their respective object, background, and aperture maps.

for i in range(0,2):#len(resamps)):
    reference=fits.open(master_file[0])
    resampled=fits.open(resamps[i])
    
    #measures the distance from the central WCS pixel to the right and to the bottom of the image, respectively.
    resdistx=(resampled[0].header['NAXIS1']-resampled[0].header['CRPIX1'])
    resdisty=(resampled[0].header['NAXIS2']-resampled[0].header['CRPIX2'])
    
    #Creates difference values that represent the start and end pixels of the resampled image on the reference image.
    ref_x_start=int(reference[0].header['CRPIX1']-resampled[0].header['CRPIX1'])
    ref_y_start=int(reference[0].header['CRPIX2']-resampled[0].header['CRPIX2'])
    ref_x_end=int(reference[0].header['CRPIX1']+resdistx)
    ref_y_end=int(reference[0].header['CRPIX2']+resdisty)
    
    #loading files into memory
    refimg=pyfits.open(master_file[0])
    resimg=pyfits.open(resamps[i])
    segimg=pyfits.open(master_segmentation[0])
    
    #converting to array
    D1=refimg[0].data
    D2=resimg[0].data
    D3=segimg[0].data
    
    #resizing and scaling pixel values
    resize_ref=D1[ref_y_start:ref_y_end,ref_x_start:ref_x_end]
    resamp_scaled=D2*resampled[0].header['FLXSCALE']
    resize_seg=D3[ref_y_start:ref_y_end,ref_x_start:ref_x_end]
    
    #doing the subtraction of the reference image from the resampled
    out_file=resamp_scaled-resize_ref
    
    #loading header information from original resampled image. This will be saved into the new images.
    head=pyfits.getheader(resamps[i])
    
    #loads the filename of the resamp image into a variable, then deletes .resamp.fits from the end of it.
    #This allows the code to write new files with the same naming format, but different file extension names.
    #If your files don't end in .resamp.fits, the -12 will need to be changed to match the number of characters
    #in the file extension.
    path=os.path.basename(resamps[i])
    new_path=path[:-12]
    
    #saves the final difference file, and the temporary resized segmentation map.
    pyfits.writeto(('./Processed_Images/'+new_path+'.difference.fits'), out_file, head)
    pyfits.writeto(('./Processed_Images/Temp/temp_segment.fits'), resize_seg, head)
    
    #produces a flag map from the temporary segmentation map.
    subprocess.call('ww -c ./config_files/default.ww -WEIGHT_NAMES '+weights[i]+',./Processed_Images/Temp/temp_segment.fits', shell=True)
    
    #does the main sextractor run on the subtracted file, producing the object, background, and aperture files.
    #this is controlled by default.sex in the config_files directory
    subprocess.call('sex ./Processed_Images/'+new_path+'.difference.fits -c ./config_files/default.sex', shell=True)
    
    #cleans up temporary files and renames background, object, and aperture files. 
    subprocess.call('rm ./Processed_Images/Temp/temp_segment.fits',shell=True)
    subprocess.call('mv ./Processed_Images/check.fits ./Processed_Images/'+new_path+'.background.fits',shell=True)
    subprocess.call('mv ./Processed_Images/check2.fits ./Processed_Images/'+new_path+'.object.fits',shell=True)
    subprocess.call('mv ./Processed_Images/check3.fits ./Processed_Images/'+new_path+'.apertures.fits',shell=True)
    subprocess.call('rm ./Processed_Images/Temp/weight.fits',shell=True)
    subprocess.call('rm ./Processed_Images/Temp/flag.fits',shell=True)
#Uncomment the next line if you would like the code to automatically clean up the master segmentation map.
#subprocess.call('rm ./Processed_Images/Temp/segmentation_map_ref.fits',shell=True)

# Begin Brendan's Transient checking script:

In [ ]:
# this is where the code can be tacked onto the pipeline, after subtracting reference images.
# At this point we should have a combined image, with reference subtracted, where the only point sources remaining
# (in theory) are transients such as supernovae or asteroids.

# This code aims to classify such sources as either known asteroids or anomalies, which can then be manually checked.

### Run SExtractor once more to spot remaining (transient) sources:

Generate list of sources:

## Check against catalogues of known asteroids:

In [ ]:
# Download latest MPCORB catalogue from:
# www.minorplanetcenter.org/iau/MPCORB/MPCORB.DAT

# Full list of paramters and how to read the data:
# http://www.minorplanetcenter.net/iau/info/MPOrbitFormat.html


# and do a manual check using PYEPHEM.

### Fiddling around with pyephem:

In [ ]:
len(perihelion_arg)

### Define a function to extract the packed Epoch date format given in the MPCORB data:

In [ ]:
# define dictionary of alpha to numbers for MPCORB packed dates translation:
# see  <http://www.minorplanetcenter.net/iau/info/PackedDates.html>  for more info.

def epoch_convert(date):
    
    "Converts the MPCORB Epoch from packed form to a regular YYYYMMDD.DDDD string to be split up and used later."
    
    packeddates = {'1':'1', '2':'2', '3':'3', '4':'4', '5':'5', '6':'6', '7':'7', '8':'8', '9':'9', \
                   '0':'0', 'A':'10', 'B':'11', 'C':'12', 'D':'13', 'E':'14', 'F':'15', 'G':'16', \
                   'H':'17', 'I':'18', 'J':'19', 'K':'20', 'L':'21', 'M':'22', 'N':'23', 'O':'24', \
                   'P':'25', 'Q':'26', 'R':'27', 'S':'28','T':'29', 'U':'30', 'V':'31' \
              }
    datestring = list(date[0:5])
    datestring2 = ""
    nums = "0123456789"
    
    # Conditionals to distinguish between packed dates, eg  avoid confusion between 1-Nov (11 1) /
    # and 11-Jan (1 11) when compiled back into a string (111). 
    # We convert all single-digit numeric month/day values to 2-digits (eg 6 --> 06):
    
    if date[3] in nums and date[4] in nums:
        datestring.insert(3,'0')
        datestring.insert(5,'0')
        for i in range(len(datestring)):
            char = packeddates['%s' % datestring[i]]
            datestring2 += str(char)
        if len(date) > 5:
            datestring2 += "." + date[5:]
        return datestring2 
    elif date[3] in nums and date[4] not in nums:
        datestring.insert(3,'0')
        for i in range(len(datestring)):
            char = packeddates['%s' % datestring[i]]
            datestring2 += str(char)
        if len(date) > 5:
            datestring2 += "." + date[5:]
        return datestring2
    elif date[3] not in nums and date[4] in nums:
        datestring.insert(4,'0')
        for i in range(len(datestring)):
            char = packeddates['%s' % datestring[i]]
            datestring2 += str(char)
        if len(date) > 5:
            datestring2 += "." + date[5:]
        return datestring2
    elif date[3] not in nums and date[4] not in nums:
        for i in range(len(datestring)):
            char = packeddates['%s' % datestring[i]]
            datestring2 += str(char)
        if len(date) > 5:
            datestring2 += "." + date[5:]
        return datestring2
    
    print(datestring2)

In [ ]:
test1 = 'K165B5566'
test2 = epoch_convert(test1)
print(test2)
hhh = (int(test2[:4]), int(test2[4:6]), float(test2[6:]))
print(hhh)
print(ephem.Date(hhh))
print('%s' % (test2))

In [ ]:
a = int(test2[:4])
b = int(test2[4:6])
c = float(test2[6:])
print(a, b, c)
print(ephem.Date((a, b, c)))
print(ephem.Date((2016, 5, 11.2566)))
ephem.Date((a, b, c))

### Load some asteroid parameters into pyephem to test:

Required orbital params:

Om longitude of the ascending node 

inc = inclination to the ecliptic (plane of the Earth's orbit)

om = argument of perihelion

a = semi-major axis, or mean distance from Sun

e = eccentricity (0=circle, 0-1=ellipse, 1=parabola)

M = mean anomaly (0 at perihelion; increases uniformly with time)

## Testing Ephemeris calculation for a real asteroid, Vesta:

In [ ]:
# initialise a new, blank orbital body and load orbital params:
params = MPCORB[2]
body = ephem.EllipticalBody()

#six required params for Keplerian orbit:
# Longitude of ascending node:
body._Om = float(params[6])
# Inclination:
body._inc = float(params[7])
# Arg of perihelion
body._om = float(params[5])
# Mean distance from Sun: ???????????? ACCURATE?? Semi Major Axis, technically.
body._a = float(params[10])
# Eccentricity:
body._e = float(params[8])
# Mean anomoly from perihelion:
body._M = float(params[4])
# Epoch for _M:
aster_epoch = epoch_convert('K161D')
#body._epoch_M = ephem.Date((int(aster_epoch[:4]), int(aster_epoch[4:6]), float(aster_epoch[6:])))
body._epoch_M = ephem.Date((2016, 01, 19))


huntsman = ephem.Observer()
huntsman.lon = 151.111128
huntsman.lat = -33.770281
huntsman.elevation = 50
huntsman.date = ephem.Date((2016, 11, 26, 0, 0, 0))
huntsman.epoch=ephem.J2000
#get fits header for date


print(body._epoch_M)
print(params)

In [ ]:
body.compute(huntsman)
print(body.a_ra, body.a_dec)
print(body.g_ra, body.g_dec)
print(body.ra, body.dec)

In [ ]:
date1 = ephem.Date((2016, 1, 19, 0, 0, 0))
body.compute(date1)
print(date1)
print(params[21])
print('Apparent Topocentric Position:      RA: ' +str(body.ra) +', DEC: '+ str(body.dec))
print('Astrometric Geocentric Position:   RA: ' +str(body.a_ra) +', DEC: '+ str(body.a_dec))
print('Apparent Geocentric Position:      RA: ' +str(body.g_ra) +', DEC: '+ str(body.g_dec))

# Correct values for 19 Jan 2016 0:00:00  as per <ssd.jpl.nasa.gov/horizons.cgi>:
# 00 52 10.62 -01 26 01.5       
 
# See what values we get for our data!

# This was tested at most recent epoch. Test further away from this to check how fast errors will accumulate.

### test reading fits header for pulling date and time of image and store as string.

In [ ]:
light = '2014-09-21_83F011167_12_light.bdfw.fits'
datestr = fits.getheader(light)['DATE']
datestr

In [ ]:
print(ephem.Date((int(datestr[0:4]), int(datestr[5:7]), int(datestr[8:10]), int(datestr[11:13]), \
                  int(datestr[14:16]), int(datestr[17:19]))))

### Can use this to calculate day fractionals:

In [ ]:
d = ephem.Date((2014, 9, 21, 15, 21, 50))
print(d.triple())

## Final code to check series of images against MPCORB: